# **Grouped `bureau_balance`**

# Data Loading and Preprocessing

27,299,925 entries.

In [11]:
from home_credit.tables import BureauBalance
from pepper.univar import print_value_counts_dict
from home_credit.utils import display_frame_basic_infos

data = BureauBalance.clean()
display_frame_basic_infos(data)
print_value_counts_dict(data, "STATUS")
display(data)

n_samples: 24 179 741
n_columns: 2, [('STATUS', 1), ('TARGET', 1)]
STATUS (7): {'C': 12006499, '0': 11799930, '1': 273013, '5': 59328, '2': 24091, '3': 9113, '4': 7767}


CLEAN_BUREAU_BALANCE                    TARGET STATUS
SK_ID_CURR SK_ID_BUREAU MONTHS_BALANCE               
100001     5896630      0                   -1      C
                        1                   -1      C
                        2                   -1      C
                        3                   -1      C
                        4                   -1      C
...                                        ...    ...
456255     5126337      72                   0      C
                        73                   0      C
                        74                   0      C
                        75                   0      0
                        76                   0      0

[24179741 rows x 2 columns]

# Key Uniqueness

We verify that there cannot be multiple `SK_ID_CURR` for one `SK_ID_BUREAU`.

The issue is therefore multi-indexed only in appearance: the `SK_ID_CURR` key is sufficient to separate the groups.

Number of `SK_ID_BUREAU` for one `SK_ID_CURR` and vice versa :

In [3]:
from home_credit.merge import _get_unique_and_multi_index, curr_prev_uniqueness_report

# Get unique and multi-indexes for the specified table and columns
indexes = _get_unique_and_multi_index(data.reset_index(), "SK_ID_BUREAU", "SK_ID_CURR")

# Generate a report on the uniqueness of SK_ID_CURR and SK_ID_BUREAU
curr_prev_uniqueness_report(*indexes)

number of unique (curr, prev)              : 774 354
number of curr with more than 1 prev       : 756 714
number of curr with one prev               : 17 640
number of curr with more than 1 prev (in %): 97.7
number of prev with more than 1 curr       : 0
number of prev with one curr               : 774 354
number of prev with more than 1 curr (in %): 0.0


# Agrégation cf. **`old_kernel_v2`**

Le premier jet était inspiré du **`lightgbm_kernel`**, un kernel de référence disponible sur Kaggle.

Il s'agit d'une agrégation par prêt qui produit 774 354 échantillons de synthèse.

L'information est appauvrie, on obtient :
- les premier, dernier et nombre de mois de suivi (96 maximum).
- les fréquences sur la période de suivi des occurrences de chaque modalité de `STATUS`.

In [4]:
from home_credit.kernel import hot_encode_cats

encoded_data, cat_vars = hot_encode_cats(data.reset_index())
months_agg_rules = {"MONTHS_BALANCE": ["min", "max", "size"]}
cat_vars_agg_rules = {col: ["mean"] for col in cat_vars}
agg_rules = months_agg_rules | cat_vars_agg_rules
aggregated = encoded_data.groupby("SK_ID_BUREAU").agg(agg_rules)
display(aggregated)

MONTHS_BALANCE           STATUS_0  STATUS_1 STATUS_2 STATUS_3  \
                        min max size      mean      mean     mean     mean   
SK_ID_BUREAU                                                                 
5001710                   0  82   83  0.421687  0.000000      0.0      0.0   
5001711                   0   3    4  1.000000  0.000000      0.0      0.0   
5001712                   0  18   19  0.526316  0.000000      0.0      0.0   
5001713                   0  21   22  1.000000  0.000000      0.0      0.0   
5001714                   0  14   15  1.000000  0.000000      0.0      0.0   
...                     ...  ..  ...       ...       ...      ...      ...   
6842884                   0  47   48  0.208333  0.000000      0.0      0.0   
6842885                   0  23   24  0.500000  0.000000      0.0      0.0   
6842886                   0  32   33  0.242424  0.000000      0.0      0.0   
6842887                   0  36   37  0.162162  0.000000      0.0      0.0   
6842888                   0  61   62  0.145161  0.016129      0.0      0.0   

             STATUS_4 STATUS_5  STATUS_C  
                 mean     mean      mean  
SK_ID_BUREAU                              
5001710           0.0      0.0  0.578313  
5001711           0.0      0.0  0.000000  
5001712           0.0      0.0  0.473684  
5001713           0.0      0.0  0.000000  
5001714           0.0      0.0  0.000000  
...               ...      ...       ...  
6842884           0.0      0.0  0.791667  
6842885           0.0      0.5  0.000000  
6842886           0.0      0.0  0.757576  
6842887           0.0      0.0  0.837838  
6842888           0.0      0.0  0.838710  

[774354 rows x 10 columns]

# Agrégation du niveau de risque (`STATUS`)

## Rappel à propos de la variable catégorielle `STATUS`

* $21\,\%$ de NA cachés sous le code `X`.
* $50\,\%$ de prêt clôturés, code `C`, ce qui est une proportion importante et pose question.
* $27\,\%$ de cas sans problème particulier.
* $12\,\%$ distribués sur 5 classes de défauts de paiement :
    * $9\,\%$ en classe `1` - Défaut maximum atteint dans les 30 derniers jours
    * $1\,\%$ en classe `2` - Défaut maximum atteint le mois précédent
    * $\varepsilon$ en classe `3` - Défaut maximum atteint il y a deux mois
    * $\varepsilon$ en classe `4` - Défaut maximum atteint il y a trois mois
    * $2\,\%$ en classe `5` - Défaut maximum atteint il y a plus de 4 mois, ou bien a été vendu, ou radié en tant que dette irrécouvrable

On lit entre les lignes qu'en cas de défaut, soit le recouvrement intervient très rapidement, le mois suivant, et la situation se régularise, ou bien qu'elle ne se régularise pas (les $\varepsilon$ sur 3 et 4).

Il est peu probable qu'un client en situation 5 obtienne à nouveau un prêt.

## Conception d'un score de risque synthétique

Pour le risque synthétique faisant abstraction du longitudinal, dans la première version, nous avons utilisé la moyenne.

C'est discutable si l'on considère ces deux aspects :
1. **comportemental** : si le client a déjà eu des incidents par le passé, a-t-il cherché à les résoudre rapidement, ou s'est-il installé dans des DPD longs.
2. **circonstanciel** : s'il y a du DPD en cours, ou récent, la santé financière actuelle du demandeur est fragile.

L'idée est donc de prendre plutôt le DPD maximum rencontré, mais avec un estompement progressif avec le temps.

Notons $s(m)$ le `STATUS` du `MONTHS_BALANCE` $m$ avec $m=0, \ldots, 95$, 0 représentant l'actuel, et 95 le plus ancien, à 4 ans d'aujourd'hui.

Les valeurs `X` et `C` sont encodées en 0.

Le niveau de risque est (voir section sur le risque d'impayé) est $e^{s(m)}$. 

Si le client a plusieurs prêts simultanés $(s_i)$, le niveau de risque du mois $m$ est $\displaystyle \sum_i e^{s_i(m)}$ et l'indice $s(m)$ agrégé est donc $\displaystyle s(m) = \log \sum e^{s_i(m)}$.

Une façon de produire un risque synthétique avec abstraction longitudinale est de diminuer l'impact de chaque terme $e^{s_i(m)}$ de la suite longitudinale par un facteur fonction de l'ancienneté.

On permet de moduler la force de ce facteur à l'aide d'un paramètre $\alpha$ par défaut à $1$ : $\displaystyle f_i(m) = \frac{e^{s_i(m)}}{1+m\alpha}$.

Ainsi, pour un défaut de paiement en cours $f_i(0) = e^{s_i(m)}$ et pour un défaut ancien, par ex. 4 ans, $\displaystyle f_i(95) = \frac{e^{s_i(95)}}{1+95\alpha}$.

La valeur moyenne du risque actualisé pour un prêt est alors :

$$\displaystyle S_i = \log \left(\frac{1}{M} \sum \frac{e^{s_i(m)}}{1+m\alpha}\right)$$

Où $M$ est le nombre de mois de suivi du client, pour un prêt particulier.

Celui du risque tous prêts confondus est alors :

$$\displaystyle S = \log \left(\frac{1}{M} \sum \frac{e^{s(m)}}{1+m\alpha}\right)$$

Où $M$ est le nombre de mois de suivi du client, tous prêts confondus.

## Agrégation par (prêt, mois) : c'est la base de fait

Simple encodage numérique de la colonne `STATUS`.

**Temps :** 3 s.

In [4]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_status_by_month()
display(status)
# ok status.info()

BUREAU_LOAN_STATUS_BY_MONTH             STATUS
SK_ID_CURR SK_ID_BUREAU MONTHS_BALANCE        
100001     5896630      0                    0
                        1                    0
                        2                    0
                        3                    0
                        4                    0
...                                        ...
456255     5126337      72                   0
                        73                   0
                        74                   0
                        75                   0
                        76                   0

[24179741 rows x 1 columns]

## Agrégation par (client, mois)

Ici, une véritable opération d'agrégation est effectuée.

On passe de 24 179 741 à 7 365 660 enregistrements.

**Temps de calcul :** 12 s.

In [4]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_status_by_client_and_month()
display(status)
# ok status.info()

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH  STATUS
SK_ID_CURR MONTHS_BALANCE                     
100001     0                              0.22
           1                              0.00
           2                              0.00
           3                              0.00
           4                              0.00
...                                        ...
456255     72                             0.00
           73                             0.00
           74                             0.00
           75                             0.00
           76                             0.00

[7365660 rows x 1 columns]

## Agrégation par prêt

Ici, on effectue la synthèse longitudinale, en conservant un indice qui représente le risque d'impayé, cf. sa dynamique sur l'ensemble des (au plus 96) mois précédents suivis.

On part de la base de fait, et on effectue l'agrégation $\displaystyle S_i = \log \left(\frac{1}{M} \sum \frac{e^{s_i(m)}}{1+m\alpha}\right)$.

Cela nous donne 774 354 scores de risque par prêt.

**Temps de calcul :** 7 s.

In [12]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_status()
display(status)
# ok status.info()

BUREAU_LOAN_STATUS,STATUS
SK_ID_BUREAU,
5001710,-2.81
5001711,-0.65
5001712,-1.68
5001713,-1.79
5001714,-1.51
...,...
6842884,-2.38
6842885,2.96
6842886,-2.09


## Agrégation par client

Là, c'est moins direct que le précédent.

Il faut partir de la pré-agrégation par (client, mois) qui effectue la synthèse mensuelle du risque tous prêts confondus.

On obtient 134 542 enregistrements.

**Temps :** 13 s.

In [14]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_status_by_client()
display(status)
# ok status.info()

BUREAU_LOAN_STATUS_BY_CLIENT,STATUS
SK_ID_CURR,
100001,-2.44
100002,-2.33
100005,-1.41
100010,-3.10
100013,-2.66
...,...
456247,-2.80
456250,-2.09
456253,-2.04


# Agrégation du nombre de prêts actifs

On commence par produire cette information au niveau de la table de base, retraitée pour imputer les NA (avec une stratégie de fill forward). Un prêt est actif si `STATUS != C`. Les `STATUS = X`, sont des NA auquel l'imputation a donné une valeur valide.

Puis on forme le nombre des prêts actifs par mois pour chaque client.

Enfin, on réalise l'abstraction longitudinale, pour chaque prêt, puis pour chaque client. Comme pour le niveau de risque, nous utilisons une moyenne amortie pour que la situation la plus récente soit mieux représentée que la plus ancienne.

## Activité mensuelle des prêts

In [3]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_activity_by_month()
display(status)
# ok status.info()

BUREAU_LOAN_ACTIVITY_BY_MONTH           ACTIVE
SK_ID_CURR SK_ID_BUREAU MONTHS_BALANCE        
100001     5896630      0                    0
                        1                    0
                        2                    0
                        3                    0
                        4                    0
...                                        ...
456255     5126337      72                   0
                        73                   0
                        74                   0
                        75                   1
                        76                   1

[24179741 rows x 1 columns]

## Nombre mensuel de prêts actifs par client

In [6]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_activity_by_client_and_month()
display(status)
# ok status.info()

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH  ACTIVE
SK_ID_CURR MONTHS_BALANCE                       
100001     0                                   3
           1                                   3
           2                                   2
           3                                   2
           4                                   2
...                                          ...
456255     72                                  1
           73                                  1
           74                                  1
           75                                  2
           76                                  2

[7365660 rows x 1 columns]

## Activité mensuelle moyenne amortie par prêt

In [2]:
from home_credit.tables import BureauBalance

status = BureauBalance.mean_loan_activity()
display(status)
# ok status.info()

BUREAU_MEAN_LOAN_ACTIVITY,SK_ID_CURR,ACTIVE
SK_ID_BUREAU,,
5001710,162368.0,0.006545
5001711,162368.0,0.520833
5001712,162368.0,0.037830
5001713,150635.0,0.167764
5001714,150635.0,0.221215
...,...,...
6842884,387020.0,0.004810
6842885,387020.0,0.157332
6842886,387020.0,0.008268


## Activité mensuelle moyenne amortie par client

In [4]:
from home_credit.tables import BureauBalance

activity = BureauBalance.mean_loan_activity_by_client()
display(activity)
# ok activity.info()

BUREAU_MEAN_LOAN_ACTIVITY_BY_CLIENT,ACTIVE
SK_ID_CURR,
100001,0.172164
100002,0.105258
100005,0.385651
100010,0.005784
100013,0.091223
...,...
456247,0.137085
456250,0.244074
456253,0.168209


# Profils denses de variation à l'aide de l'encodage RLE

Il se peut que nous ayons besoin des informations longitudinales, soit brutes, soit agrégées (état, dénombrement, montant). Cela pourra servir d'abord au niveau des fusions avec les tables **`bureau`** et **`application`** si l'on choisit d'essayer une version à 96 dimensions d'une caractéristique ou d'une autre dont l'importance serait absolument déterminante (il faudra alors introduire une étape de réduction de dimensionnalité). Cela pourra également servir pour des agrégations transversales, pour un demandeur donné, entre les informations provenant de **`bureau`** et d'autres, de même nature, provenant de **`previous_application`**

Un pivotement entraînement l'apparition d'une quantité importante de cellules vides et nous forcerait à passer une gestion de matrices creuses, ce qui ajouterait une couche de complexité au code.

Pour pouvoir sauvegarder ces informations de manière dense, nous avons choisi d'utiliser la technique classique de compression RLE. Elle nous permet de condenser la suite de valeurs mensuelles d'un caractéristique, qui souvent reste constante sur l'ensemble de la période ou sur de longues sous-périodes.

Une relation de passage inverse nous permet de développer un vecteur de séquences codées RLE en un dataframe multi-dimensionnel qui serait le résultat d'un pivotement.

Il important de souligner que de nombreuses opérations (dériver un indicateur, union, intersection, etc) de suites peuvent être effectuée entre codes RLE sans repasser par une version développée.

Dans le cadre de la table **`bureau_balance`** chaque information que nous avons dégagée au grain mensuel peut être codifiée.

## Période de suivi

La période de suivi d'un prêt peut être la totalité des 96 mois, soit une sous-période, voire des sous-périodes fragmentées.

Notre première fonction, basée sur la fonction `jumps_rle` permet de codifier en RLE les sauts entre mois consécutifs de suivi.

### Période de suivi par prêt

On produit la table indexée par `SK_ID_BUREAU` des périodes de suivi des prêts, avec le premier, le dernier et le nombre de mois de suivi, et la représentation RLE des sous-périodes (dans la plupart des cas, une seule).

**Time :** 1 m.

In [4]:
from home_credit.tables import BureauBalance

tracking = BureauBalance.rle_loan_tracking_period()
display(tracking)
# ok tracking.info()

CLEAN_BUREAU_BALANCE MONTHS_BALANCE                     
                                min max count  jumps_rle
SK_ID_BUREAU                                            
5001710                           0  82    83  [[1, 83]]
5001711                           0   3     4   [[1, 4]]
5001712                           0  18    19  [[1, 19]]
5001713                           0  21    22  [[1, 22]]
5001714                           0  14    15  [[1, 15]]
...                             ...  ..   ...        ...
6842884                           0  47    48  [[1, 48]]
6842885                           0  23    24  [[1, 24]]
6842886                           0  32    33  [[1, 33]]
6842887                           0  36    37  [[1, 37]]
6842888                           0  61    62  [[1, 62]]

[774354 rows x 4 columns]

### Période de suivi par client

On produit la table indexée par `SK_ID_CURR` des périodes de suivi, tous prêts confondus, avec le premier, le dernier et le nombre de mois de suivi, et la représentation RLE des sous-périodes (dans la plupart des cas, une seule).

Exemple pour aider à l'interprétation des codes RLE :

```
100010	2	90	72	((3, 1), (1, 35), (18, 1), (1, 35))
```

Indique une période courant les mois 2 à 90, mais avec 72 mois effectivement suivis, avec une première période de 36 mois, un _gap_ de de 18 mois, puis une nouvelle période de suivi de 36 mois.

Application : sur 134 542 suivis, 113 748 le sont sans interruption jusqu'à aujourd'hui.

**Time :** 22 s.

In [5]:
from home_credit.tables import BureauBalance

tracking = BureauBalance.rle_loan_tracking_period_by_client()
display(tracking)
# ok status.info()

CLEAN_BUREAU_BALANCE MONTHS_BALANCE            \
                                min max count   
SK_ID_CURR                                      
100001                            0  51    52   
100002                            0  47    48   
100005                            0  12    13   
100010                            2  90    72   
100013                            0  68    69   
...                             ...  ..   ...   
456247                            0  81    82   
456250                            0  32    33   
456253                            0  30    31   
456254                            0  36    37   
456255                            0  76    77   

CLEAN_BUREAU_BALANCE                                       
                                                jumps_rle  
SK_ID_CURR                                                 
100001                                          [[1, 52]]  
100002                                          [[1, 48]]  
100005                                          [[1, 13]]  
100010                [[3, 1], [1, 35], [18, 1], [1, 35]]  
100013                                          [[1, 69]]  
...                                                   ...  
456247                                          [[1, 82]]  
456250                                          [[1, 33]]  
456253                                          [[1, 31]]  
456254                                          [[1, 37]]  
456255                                          [[1, 77]]  

[134542 rows x 4 columns]

Exemple d'utilisation : nombre de périodes de suivi continues jusqu'à aujourd'hui :

In [6]:
rle_month_jumps = tracking[("MONTHS_BALANCE", "jumps_rle")]
is_continuous_tracking = rle_month_jumps.apply(lambda x: len(x) == 1)

print(f"# continuous tracking periods : {sum(is_continuous_tracking)}")

# continuous tracking periods : 113748


## Variations longitudinales (mensuelles)

Pour être interprétées, elles doivent toujours être associées à une période de suivi support.

### Variation d'activité par prêt

**Temps :** 4 m 15 s.

In [4]:
from home_credit.tables import BureauBalance

activity_variation = BureauBalance.rle_loan_activity_variation()
display(activity_variation)
# ok activity_variation.info()

BUREAU_LOAN_ACTIVITY_BY_MONTH MONTHS_BALANCE                       \
                                         min max count  jumps_rle   
SK_ID_BUREAU                                                        
5001710                                    0  82    83  [[1, 83]]   
5001711                                    0   3     4   [[1, 4]]   
5001712                                    0  18    19  [[1, 19]]   
5001713                                    0  21    22  [[1, 22]]   
5001714                                    0  14    15  [[1, 15]]   
...                                      ...  ..   ...        ...   
6842884                                    0  47    48  [[1, 48]]   
6842885                                    0  23    24  [[1, 24]]   
6842886                                    0  32    33  [[1, 33]]   
6842887                                    0  36    37  [[1, 37]]   
6842888                                    0  61    62  [[1, 62]]   

BUREAU_LOAN_ACTIVITY_BY_MONTH               ACTIVE  
                              series_rle_reduction  
SK_ID_BUREAU                                        
5001710                         [[0, 48], [1, 35]]  
5001711                                   [[1, 4]]  
5001712                          [[0, 9], [1, 10]]  
5001713                                  [[1, 22]]  
5001714                                  [[1, 15]]  
...                                            ...  
6842884                         [[0, 38], [1, 10]]  
6842885                                  [[1, 24]]  
6842886                          [[0, 25], [1, 8]]  
6842887                          [[0, 31], [1, 6]]  
6842888                         [[0, 52], [1, 10]]  

[774354 rows x 5 columns]

### Variation d'activité par client

**Temps :** 45 s.

In [3]:
from home_credit.tables import BureauBalance

activity_variation = BureauBalance.rle_loan_activity_by_client_variation()
display(activity_variation)
# ok activity_variation.info()

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH MONTHS_BALANCE            \
                                                    min max count   
SK_ID_CURR                                                          
100001                                                0  51    52   
100002                                                0  47    48   
100005                                                0  12    13   
100010                                                2  90    72   
100013                                                0  68    69   
...                                                 ...  ..   ...   
456247                                                0  81    82   
456250                                                0  32    33   
456253                                                0  30    31   
456254                                                0  36    37   
456255                                                0  76    77   

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH                                       \
                                                                    jumps_rle   
SK_ID_CURR                                                                      
100001                                                              [[1, 52]]   
100002                                                              [[1, 48]]   
100005                                                              [[1, 13]]   
100010                                    [[3, 1], [1, 35], [18, 1], [1, 35]]   
100013                                                              [[1, 69]]   
...                                                                       ...   
456247                                                              [[1, 82]]   
456250                                                              [[1, 33]]   
456253                                                              [[1, 31]]   
456254                                                              [[1, 37]]   
456255                                                              [[1, 77]]   

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH                                             ACTIVE  
                                                                       series_rle_reduction  
SK_ID_CURR                                                                                   
100001                                    [[3, 2], [2, 9], [1, 10], [2, 8], [1, 1], [0, ...  
100002                                    [[1, 4], [0, 9], [1, 3], [0, 1], [1, 1], [2, 3...  
100005                                                                    [[2, 3], [1, 10]]  
100010                                                 [[0, 26], [1, 10], [0, 26], [1, 10]]  
100013                                    [[1, 18], [2, 22], [1, 1], [2, 3], [3, 11], [2...  
...                                                                                     ...  
456247                                    [[2, 1], [3, 2], [4, 7], [3, 1], [2, 1], [1, 1...  
456250                                                    [[2, 25], [3, 1], [2, 2], [1, 5]]  
456253                                                            [[1, 19], [4, 5], [3, 7]]  
456254                                                                    [[0, 29], [1, 8]]  
456255                                    [[0, 3], [2, 4], [3, 3], [4, 2], [3, 1], [2, 2...  

[134542 rows x 5 columns]

### Variation du niveau de risque par prêt

**Temps :** 4 m.

In [9]:
from home_credit.tables import BureauBalance

status_variation = BureauBalance.rle_loan_status_variation()
display(status_variation)
# ok status_variation.info()

BUREAU_LOAN_STATUS_BY_MONTH MONTHS_BALANCE                       \
                                       min max count  jumps_rle   
SK_ID_BUREAU                                                      
5001710                                  0  82    83  [[1, 83]]   
5001711                                  0   3     4   [[1, 4]]   
5001712                                  0  18    19  [[1, 19]]   
5001713                                  0  21    22  [[1, 22]]   
5001714                                  0  14    15  [[1, 15]]   
...                                    ...  ..   ...        ...   
6842884                                  0  47    48  [[1, 48]]   
6842885                                  0  23    24  [[1, 24]]   
6842886                                  0  32    33  [[1, 33]]   
6842887                                  0  36    37  [[1, 37]]   
6842888                                  0  61    62  [[1, 62]]   

BUREAU_LOAN_STATUS_BY_MONTH                     STATUS  
                                  series_rle_reduction  
SK_ID_BUREAU                                            
5001710                                      [[0, 83]]  
5001711                                       [[0, 4]]  
5001712                                      [[0, 19]]  
5001713                                      [[0, 22]]  
5001714                                      [[0, 15]]  
...                                                ...  
6842884                                      [[0, 48]]  
6842885                             [[5, 12], [0, 12]]  
6842886                                      [[0, 33]]  
6842887                                      [[0, 37]]  
6842888                      [[0, 58], [1, 1], [0, 3]]  

[774354 rows x 5 columns]

### Variation du niveau de risque par client

In [8]:
from home_credit.tables import BureauBalance

status_variation = BureauBalance.rle_loan_status_by_client_variation()
display(status_variation)
# ok status_variation.info()

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH MONTHS_BALANCE            \
                                                  min max count   
SK_ID_CURR                                                        
100001                                              0  51    52   
100002                                              0  47    48   
100005                                              0  12    13   
100010                                              2  90    72   
100013                                              0  68    69   
...                                               ...  ..   ...   
456247                                              0  81    82   
456250                                              0  32    33   
456253                                              0  30    31   
456254                                              0  36    37   
456255                                              0  76    77   

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH                                       \
                                                                  jumps_rle   
SK_ID_CURR                                                                    
100001                                                            [[1, 52]]   
100002                                                            [[1, 48]]   
100005                                                            [[1, 13]]   
100010                                  [[3, 1], [1, 35], [18, 1], [1, 35]]   
100013                                                            [[1, 69]]   
...                                                                     ...   
456247                                                            [[1, 82]]   
456250                                                            [[1, 33]]   
456253                                                            [[1, 31]]   
456254                                                            [[1, 37]]   
456255                                                            [[1, 77]]   

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH                                             STATUS  
                                                                     series_rle_reduction  
SK_ID_CURR                                                                                 
100001                                           [[0.2199999988079071, 1.0], [0.0, 51.0]]  
100002                                  [[0.0, 19.0], [0.36000001430511475, 1.0], [0.6...  
100005                                                                      [[0.0, 13.0]]  
100010                                                                      [[0.0, 72.0]]  
100013                                  [[0.0, 18.0], [0.36000001430511475, 1.0], [0.0...  
...                                                                                   ...  
456247                                                                      [[0.0, 82.0]]  
456250                                                                      [[0.0, 33.0]]  
456253                                                                      [[0.0, 31.0]]  
456254                                                                      [[0.0, 37.0]]  
456255                                                                      [[0.0, 77.0]]  

[134542 rows x 5 columns]